# ПРОГНОЗ ОТТОКА ПОЛЬЗОВАТЕЛЕЙ КРЕДИТНЫХ КАРТ

## ЗАДАЧА
Необходимо построить прогнозную модель, которая сможет определить, перестанет ли клиент банка пользоваться кредитной картой.

#### В таблице представлены следующие столбцы:

**CLIENTNUM** - Номер клиента. Уникальный идентификатор клиента, которому принадлежит счет

**Attrition_Flag** - Aктивность клиента — если учетная запись закрыта, то Attrited Customer, иначе Existing Customer

**Customer_Age** - Bозраст клиента в годах

**Gender** - М=мужчина, Ж=женщина

**Dependent_count** - Количество иждивенцев

**Education_Level** - Образование, квалификация владельца счета (пример: средняя школа, выпускник колледжа и т. д.)

**Marital_Status** - Женат, Холост, Разведен, Неизвестно

**Income_Category** - Категория годового дохода владельца счета (< 40 000 долларов США, 40 000-60 000 долларов США, 60 000-80 000 долларов США, 80 000-120 000 долларов США, >

**Card_Category** - Тип карты (Синяя, Серебряная, Золотая, Платиновая)

**Months_on_book** - Период отношений с банком

**Total_Relationship_Count** - Общее количество продукции, хранящейся у клиента

**Months_Inactive_12_mon** - Количество месяцев бездействия за последние 12 месяцев

**Contacts_Count_12_mon** - Количество контактов за последние 12 месяцев

**Credit_Limit** - Кредитный лимит по кредитной карте

**Total_Revolving_Bal** - Общий оборотный баланс на кредитной карте

**Avg_Open_To_Buy** - Открытая кредитная линия для покупки (среднее значение за последние 12 месяцев)

**Total_Amt_Chng_Q4_Q1** - Изменение суммы транзакции (Q4 по сравнению с Q1)

**Total_Trans_Amt** - Общая сумма транзакции (последние 12 месяцев)

**Total_Trans_Ct** - Общее количество транзакций (последние 12 месяцев)

**Total_Ct_Chng_Q4_Q1** - Изменение количества транзакций (Q4 по сравнению с Q1)

**Avg_Utilization_Ratio** - Средний коэффициент использования карт

### Целевой признак
**Attrition_Flag** - Aктивность клиента — если учетная запись закрыта, то Attrited Customer, иначе Existing Customer

**МЕТРИКА**

Вычислите оценку F1, также известную как сбалансированная F-оценка или F-мера.

Показатель F1 можно интерпретировать как среднее гармоническое между точностью и отзывом, где показатель F1 достигает своего наилучшего значения при 1, а наихудший показатель при 0. Относительный вклад точности и отзыва в показатель F1 равен.

Формула для оценки F1:

F1 = 2 * (precision * recall) / (precision + recall)

In [277]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, roc_curve, auc,
                             classification_report)

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import RocCurveDisplay
from torchmetrics import F1Score

import matplotlib.pyplot as plt
%matplotlib inline

In [136]:
train_df = pd.read_csv('train.csv')

In [137]:
train_df.head()

,CLIENTNUM,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,...,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Attrition_Flag
0,715630983,31,F,0,Graduate,Single,Less than $40K,Blue,21,4,...,3,4598.0,0,4598.0,0.439,6317,77,0.833,0.000,Attrited Customer
1,713643858,53,F,1,College,Married,$40K - $60K,Blue,43,4,...,2,1525.0,1411,114.0,0.660,1911,47,0.958,0.925,Existing Customer
2,708334158,46,F,3,Graduate,Divorced,Less than $40K,Blue,34,4,...,4,9863.0,686,9177.0,0.581,3068,58,0.933,0.070,Existing Customer
3,711033408,50,F,2,High School,Married,Less than $40K,Blue,36,5,...,2,9355.0,1626,7729.0,0.701,1230,28,0.867,0.174,Existing Customer
4,711368283,39,M,2,Unknown,Married,$80K - $120K,Blue,27,3,...,2,34516.0,1629,32887.0,1.078,3448,53,1.038,0.047,Existing Customer


In [138]:
train_df.tail()

,CLIENTNUM,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,...,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Attrition_Flag
7590,772596783,31,M,1,Graduate,Unknown,Less than $40K,Gold,21,3,...,2,15261.0,1109,14152.0,0.655,15442,125,0.761,0.073,Existing Customer
7591,820330008,46,M,5,High School,Married,$120K +,Blue,42,1,...,3,32926.0,1540,31386.0,0.862,8231,98,0.556,0.047,Existing Customer
7592,720614658,40,M,2,High School,Divorced,$60K - $80K,Blue,25,6,...,2,2978.0,0,2978.0,0.490,3879,95,0.583,0.000,Existing Customer
7593,709759908,46,M,2,Graduate,Married,$60K - $80K,Blue,36,4,...,2,1939.0,685,1254.0,0.421,4327,84,0.680,0.353,Existing Customer
7594,814489683,34,F,2,Unknown,Single,Less than $40K,Blue,29,3,...,0,5387.0,795,4592.0,0.837,2596,59,1.034,0.148,Existing Customer


In [139]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7595 entries, 0 to 7594
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   CLIENTNUM                 7595 non-null   int64  
 1   Customer_Age              7595 non-null   int64  
 2   Gender                    7595 non-null   object 
 3   Dependent_count           7595 non-null   int64  
 4   Education_Level           7595 non-null   object 
 5   Marital_Status            7595 non-null   object 
 6   Income_Category           7595 non-null   object 
 7   Card_Category             7595 non-null   object 
 8   Months_on_book            7595 non-null   int64  
 9   Total_Relationship_Count  7595 non-null   int64  
 10  Months_Inactive_12_mon    7595 non-null   int64  
 11  Contacts_Count_12_mon     7595 non-null   int64  
 12  Credit_Limit              7595 non-null   float64
 13  Total_Revolving_Bal       7595 non-null   int64  
 14  Avg_Open

In [140]:
train_df.Attrition_Flag.value_counts()

Existing Customer    6375
Attrited Customer    1220
Name: Attrition_Flag, dtype: int64

Заменим значения целевого признака на:

    0 - Attrited Customer
    1 - Existing Customer

In [141]:
train_df['Attrition_Flag'] = train_df['Attrition_Flag'].apply(lambda x: 1 if x == 'Attrited Customer' else 0)

In [142]:
train_df.Attrition_Flag.value_counts()

0    6375
1    1220
Name: Attrition_Flag, dtype: int64

In [143]:
train_df[train_df['Attrition_Flag'] == 0]['Gender'].value_counts(normalize=True)

F    0.521255
M    0.478745
Name: Gender, dtype: float64

In [144]:
numeric_features = []
object_features = []
for col in train_df.columns:
    if train_df[col].dtype == 'object':
        object_features.append(col)
    else:
        numeric_features.append(col)

In [145]:
train_df['Gender'] = train_df['Gender'].apply(lambda x: 0 if x == 'M' else 1)

In [146]:
train_df.Education_Level.value_counts()

Graduate         2322
High School      1518
Unknown          1169
Uneducated       1126
College           752
Post-Graduate     373
Doctorate         335
Name: Education_Level, dtype: int64

In [147]:
train_df.groupby('Education_Level', as_index=False)['Attrition_Flag'].mean()

,Education_Level,Attrition_Flag
0,College,0.147606
1,Doctorate,0.200000
2,Graduate,0.151163
3,High School,0.159420
4,Post-Graduate,0.184987
5,Uneducated,0.158082
6,Unknown,0.172797


In [148]:
train_df.Marital_Status.value_counts()

Married     3480
Single      2987
Divorced     574
Unknown      554
Name: Marital_Status, dtype: int64

In [149]:
train_df.groupby('Marital_Status', as_index=False)['Attrition_Flag'].mean()

,Marital_Status,Attrition_Flag
0,Divorced,0.175958
1,Married,0.147701
2,Single,0.171075
3,Unknown,0.169675


In [150]:
train_df.Income_Category.value_counts()

Less than $40K    2675
$40K - $60K       1309
$80K - $120K      1160
$60K - $80K       1047
Unknown            846
$120K +            558
Name: Income_Category, dtype: int64

In [151]:
train_df.groupby('Income_Category', as_index=False)['Attrition_Flag'].mean()

,Income_Category,Attrition_Flag
0,$120K +,0.177419
1,$40K - $60K,0.153552
2,$60K - $80K,0.147087
3,$80K - $120K,0.149138
4,Less than $40K,0.169720
5,Unknown,0.164303


In [152]:
train_df.Card_Category.value_counts()

Blue        7065
Silver       423
Gold          91
Platinum      16
Name: Card_Category, dtype: int64

In [153]:
train_df.groupby('Card_Category', as_index=False)['Attrition_Flag'].mean()

,Card_Category,Attrition_Flag
0,Blue,0.160085
1,Gold,0.208791
2,Platinum,0.187500
3,Silver,0.158392


In [154]:
train_df.iloc[:, 1:].corr()

,Customer_Age,Gender,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Attrition_Flag
Customer_Age,1.000000,0.013837,-0.126147,0.784460,-0.021288,0.045027,-0.026055,0.004731,0.011376,0.003714,-0.064638,-0.052881,-0.069429,-0.013271,-0.000643,0.019546
Gender,0.013837,1.000000,-0.005803,-0.001703,-0.000776,0.014930,-0.045534,-0.414193,-0.023668,-0.412113,-0.019812,-0.016732,0.076735,0.009620,0.261269,0.030789
Dependent_count,-0.126147,-0.005803,1.000000,-0.106327,-0.040563,-0.014897,-0.032803,0.061806,-0.002262,0.062013,-0.033297,0.017813,0.040228,0.006256,-0.038256,0.020994
Months_on_book,0.784460,-0.001703,-0.106327,1.000000,-0.013214,0.069374,-0.019347,0.015324,0.008727,0.014545,-0.045591,-0.044868,-0.053434,-0.013663,-0.017735,0.013759
Total_Relationship_Count,-0.021288,-0.000776,-0.040563,-0.013214,1.000000,-0.016609,0.050970,-0.078688,0.028378,-0.081233,0.049162,-0.351418,-0.243580,0.044704,0.083213,-0.151227
Months_Inactive_12_mon,0.045027,0.014930,-0.014897,0.069374,-0.016609,1.000000,0.032637,-0.023829,-0.039596,-0.020291,-0.031013,-0.039610,-0.045135,-0.037613,-0.007469,0.153219
Contacts_Count_12_mon,-0.026055,-0.045534,-0.032803,-0.019347,0.050970,0.032637,1.000000,0.023844,-0.061323,0.029329,-0.029729,-0.106406,-0.154010,-0.103147,-0.058205,0.206200
Credit_Limit,0.004731,-0.414193,0.061806,0.015324,-0.078688,-0.023829,0.023844,1.000000,0.045661,0.996003,0.020030,0.168750,0.069019,-0.001019,-0.482334,-0.024973
Total_Revolving_Bal,0.011376,-0.023668,-0.002262,0.008727,0.028378,-0.039596,-0.061323,0.045661,1.000000,-0.043743,0.052979,0.066880,0.066615,0.083246,0.619816,-0.270251
Avg_Open_To_Buy,0.003714,-0.412113,0.062013,0.014545,-0.081233,-0.020291,0.029329,0.996003,-0.043743,1.000000,0.015295,0.162785,0.063069,-0.008462,-0.537792,-0.000813


In [155]:
df = train_df[numeric_features]

In [156]:
X = train_df[numeric_features].iloc[:, 1:-1].values

In [157]:
y = train_df[numeric_features].iloc[:, [-1]].values#.reshape(1, -1)

In [158]:
# y_pred = np.ones(X_test.shape[0],).reshape(-1, 1)

In [159]:
# scaler = StandardScaler()
scaler = MinMaxScaler()

In [160]:
X = scaler.fit_transform(X)

In [161]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=0)

In [162]:
model = LogisticRegression()
model.fit(X_train, y_train.ravel())
y_pred = model.predict(X_test)
f1_score(y_test, y_pred)

0.619047619047619

In [171]:
model = SVC(C=6)
model.fit(X_train, y_train.ravel())
y_pred = model.predict(X_test)
f1_score(y_test, y_pred)

0.810344827586207

In [172]:
model = GaussianNB()
model.fit(X_train, y_train.ravel())
y_pred = model.predict(X_test)
f1_score(y_test, y_pred)

0.627906976744186

In [177]:
model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train, y_train.ravel())
y_pred = model.predict(X_test)
f1_score(y_test, y_pred)


0.6951672862453533

In [178]:
import torch
from torch.utils.data import Dataset, DataLoader

In [179]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [249]:
class MyDataset(Dataset):
    def __init__(self, X, y=None):
        self.n_samples = X.shape[0]
        self.x_data = torch.from_numpy(X).type(torch.float32)
        if not y is None:
            self.y_data = torch.from_numpy(y).type(torch.int64)
        else:
            self.y_data = torch.zeros(self.n_samples).type(torch.float)

    
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.n_samples

In [250]:
# Класс - модуль, реализующий архитектуру. Можно было бы использовать nn.Sequential
class FeedforwardNeuralNetModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Первый полносвязный слой
        self.fc1 = torch.nn.Linear(input_dim, hidden_dim, device=device) 

        # Нелинейная функция
        self.activation = torch.nn.ReLU()
#         self.activation = torch.nn.Tanh()
        
        
        # Полносвязный слой
        self.fc2 = torch.nn.Linear(hidden_dim, output_dim, device=device)  

    def forward(self, x):
        # Полносвязный слой
        out = self.fc1(x)

        # Нелинейная функция
        out = self.activation(out)

        # Полносвязный слой
        out = self.fc2(out)
        
        return out

In [300]:
batch_size = 100
n_iters = 100000

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\evstr\anaconda3\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\evstr\anaconda3\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] Удаленный хост принудительно разорвал существующее подключение
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\evstr\anaconda3\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\evstr\anaconda3\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] Удаленный хост принудительно разорвал сущес

In [301]:
input_dim = X.shape[1]
hidden_dim = 300
output_dim = 2

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\evstr\anaconda3\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\evstr\anaconda3\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] Удаленный хост принудительно разорвал существующее подключение
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\evstr\anaconda3\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\evstr\anaconda3\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] Удаленный хост принудительно разорвал сущес

In [302]:
criterion = torch.nn.CrossEntropyLoss()

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\evstr\anaconda3\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\evstr\anaconda3\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] Удаленный хост принудительно разорвал существующее подключение
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\evstr\anaconda3\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\evstr\anaconda3\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] Удаленный хост принудительно разорвал сущес

In [294]:
learning_rate = 0.1

In [295]:
# y = y.reshape(-1)

In [296]:
model = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)

In [297]:
train_dataset.y_data.shape

torch.Size([5696, 1])

In [298]:
f1 = F1Score(num_classes=2)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True)

train_dataset = MyDataset(X_train, y_train)
test_dataset = MyDataset(X_test, y_test)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

# Передаем параметры в оптимизатор, чтобы для них считались градиенты
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
iteration = 0

for epoch in range(num_epochs):
    for i, (X_, y_) in enumerate(train_loader):
#         # Спрямляем изображение в вектор
# #         images = images.view(-1, 28*28)
# #         images = images.cuda(device)
# #         labels = labels.cuda(device)
#         y_ = y_[0]
        y_ = y_.reshape(-1)

        # Затираем прошлые градиенты, чтобы они не аккумулировались
        optimizer.zero_grad()

        # Получаем выход модели
        outputs = model(X_)
        
#         # Вычисляем значение функции потерь
        loss = criterion(outputs, y_)
                

#         # Здесь происходит вычисление градентов для параметров модели, которые переданы в оптимизатор
        loss.backward()
# #         break
        
#         # Здесь происходит обновление параметров
        optimizer.step()

        iteration += 1
        
# #         break
        if iteration % 1000 == 0:
            # Вычисляем точность (можно пользоваться и sklearn)         
            correct = 0
            total = 0
            # Итерация по всему валидационному датасету, "переключаем" модель в режим инференса
            # (в данном случае не обязательно, но в общем случае важно не забывать)
            model.eval()
            
            # Поскольку нам не нужно вычисление градиентов, мы используем менеджер контекста
            with torch.no_grad():
                for X_, y_ in test_loader:
                    y_ = y_.reshape(-1)
# #                     y_ = y_[0]
#                     # Преобразуем изображение в вектор
# #                     images = images.view(-1, 28*28)
                    
# #                     images = images.cuda(device)
# #                     labels = labels.cuda(device)
                    
                    # Получаем выход модели
                    outputs = model(X_)
                    
                    # Берем класс, имеющий наибольшую вероятность (здесь логиты)
                    _, predicted = torch.max(outputs.data, 1)
# #                     predicted = outputs
# 
                    # Подсчитываем общее количество объектов в тестовой выборке
                    total += y_.size(0)

                    # Общее количество верно классифицированных объектов
#                     correct += (predicted == y_).sum()
                    correct += f1(predicted, y_)
                model.train()
            accuracy = 100 * correct / total
            
#             print('Итерация: {}. Validation loss: {}. Validation accuracy: {}'.format(iteration, loss.item(), accuracy))
            print('Итерация: {}. Validation loss: {}. Validation f1_score: {}'.format(iteration, loss.item(), accuracy))

Итерация: 1000. Validation loss: 0.20475484430789948. Validation f1_score: 0.9077929854393005
Итерация: 2000. Validation loss: 0.18492411077022552. Validation f1_score: 0.910947322845459
Итерация: 3000. Validation loss: 0.2554430663585663. Validation f1_score: 0.9167503118515015
Итерация: 4000. Validation loss: 0.29328736662864685. Validation f1_score: 0.9172877669334412
Итерация: 5000. Validation loss: 0.3411825895309448. Validation f1_score: 0.9130590558052063
Итерация: 6000. Validation loss: 0.15928569436073303. Validation f1_score: 0.9288780093193054
Итерация: 7000. Validation loss: 0.1637565791606903. Validation f1_score: 0.9315109252929688
Итерация: 8000. Validation loss: 0.13797390460968018. Validation f1_score: 0.9341492652893066
Итерация: 9000. Validation loss: 0.1021001935005188. Validation f1_score: 0.9320374727249146
Итерация: 10000. Validation loss: 0.19514425098896027. Validation f1_score: 0.9357237815856934
Итерация: 11000. Validation loss: 0.19565193355083466. Validatio

Итерация: 88000. Validation loss: 0.05131978541612625. Validation f1_score: 0.9473194479942322
Итерация: 89000. Validation loss: 0.08512142300605774. Validation f1_score: 0.9288780093193054
Итерация: 90000. Validation loss: 0.0927799716591835. Validation f1_score: 0.9462555646896362
Итерация: 91000. Validation loss: 0.03552917763590813. Validation f1_score: 0.9499524831771851
Итерация: 92000. Validation loss: 0.08278456330299377. Validation f1_score: 0.9441545009613037
Итерация: 93000. Validation loss: 0.07519988715648651. Validation f1_score: 0.9352076649665833
Итерация: 94000. Validation loss: 0.061787690967321396. Validation f1_score: 0.9504789113998413
Итерация: 95000. Validation loss: 0.06531862169504166. Validation f1_score: 0.9462609887123108
Итерация: 96000. Validation loss: 0.06362730264663696. Validation f1_score: 0.9373087882995605
Итерация: 97000. Validation loss: 0.14340953528881073. Validation f1_score: 0.9352025985717773
Итерация: 98000. Validation loss: 0.12440956383943

Итерация: 174000. Validation loss: 0.04745478183031082. Validation f1_score: 0.9467874765396118
Итерация: 175000. Validation loss: 0.016128145158290863. Validation f1_score: 0.9462608695030212
Итерация: 176000. Validation loss: 0.04387359693646431. Validation f1_score: 0.9494204521179199
Итерация: 177000. Validation loss: 0.027491163462400436. Validation f1_score: 0.9504736065864563
Итерация: 178000. Validation loss: 0.009807765483856201. Validation f1_score: 0.9483672976493835
Итерация: 179000. Validation loss: 0.033582109957933426. Validation f1_score: 0.9483726620674133
Итерация: 180000. Validation loss: 0.07059960812330246. Validation f1_score: 0.9315110445022583
Итерация: 181000. Validation loss: 0.016226040199398994. Validation f1_score: 0.9452076554298401
Итерация: 182000. Validation loss: 0.015997886657714844. Validation f1_score: 0.9467874765396118
Итерация: 183000. Validation loss: 0.04175715520977974. Validation f1_score: 0.9420482516288757
Итерация: 184000. Validation loss:

Итерация: 259000. Validation loss: 0.006958444602787495. Validation f1_score: 0.9473140835762024
Итерация: 260000. Validation loss: 0.020837996155023575. Validation f1_score: 0.9467821717262268
Итерация: 261000. Validation loss: 0.010166394524276257. Validation f1_score: 0.9462608695030212
Итерация: 262000. Validation loss: 0.010120531544089317. Validation f1_score: 0.9478353261947632
Итерация: 263000. Validation loss: 0.018719909712672234. Validation f1_score: 0.9467874765396118
Итерация: 264000. Validation loss: 0.011109389364719391. Validation f1_score: 0.9462608695030212
Итерация: 265000. Validation loss: 0.008194881491363049. Validation f1_score: 0.9483672976493835
Итерация: 266000. Validation loss: 2.7937653064727783. Validation f1_score: 0.8124211430549622
Итерация: 267000. Validation loss: 0.010344925336539745. Validation f1_score: 0.9462608695030212
Итерация: 268000. Validation loss: 0.007582717575132847. Validation f1_score: 0.9473087787628174
Итерация: 269000. Validation los

Итерация: 344000. Validation loss: 0.003267099615186453. Validation f1_score: 0.9452076554298401
Итерация: 345000. Validation loss: 0.008256248198449612. Validation f1_score: 0.9467874765396118
Итерация: 346000. Validation loss: 0.004530702251940966. Validation f1_score: 0.9473141431808472
Итерация: 347000. Validation loss: 0.01091749593615532. Validation f1_score: 0.9499416351318359
Итерация: 348000. Validation loss: 0.002504058415070176. Validation f1_score: 0.9446810483932495
Итерация: 349000. Validation loss: 0.00617173733189702. Validation f1_score: 0.948361873626709
Итерация: 350000. Validation loss: 0.003941433038562536. Validation f1_score: 0.9462609887123108
Итерация: 351000. Validation loss: 0.003923849668353796. Validation f1_score: 0.9488938450813293
Итерация: 352000. Validation loss: 0.0058923643082380295. Validation f1_score: 0.9446811079978943
Итерация: 353000. Validation loss: 0.014235353097319603. Validation f1_score: 0.948361873626709
Итерация: 354000. Validation loss

...

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\evstr\anaconda3\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\evstr\anaconda3\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] Удаленный хост принудительно разорвал существующее подключение
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\evstr\anaconda3\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\evstr\anaconda3\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] Удаленный хост принудительно разорвал сущес

Итерация: 419000. Validation loss: 0.0029787786770612. Validation f1_score: 0.9473140835762024
Итерация: 420000. Validation loss: 0.005860463250428438. Validation f1_score: 0.9473140835762024
Итерация: 421000. Validation loss: 0.003719164989888668. Validation f1_score: 0.9494150876998901
Итерация: 422000. Validation loss: 0.003813238814473152. Validation f1_score: 0.9488937854766846
Итерация: 423000. Validation loss: 0.0003892497334163636. Validation f1_score: 0.9483672976493835
Итерация: 424000. Validation loss: 0.0064143524505198. Validation f1_score: 0.9473085999488831
Итерация: 425000. Validation loss: 0.004749658517539501. Validation f1_score: 0.9473140835762024
Итерация: 426000. Validation loss: 0.0024650872219353914. Validation f1_score: 0.9446811079978943
Итерация: 427000. Validation loss: 0.0023321944754570723. Validation f1_score: 0.9483672976493835
Итерация: 428000. Validation loss: 0.003086692653596401. Validation f1_score: 0.9467874765396118
Итерация: 429000. Validation lo

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\evstr\anaconda3\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\evstr\anaconda3\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] Удаленный хост принудительно разорвал существующее подключение
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\evstr\anaconda3\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\evstr\anaconda3\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] Удаленный хост принудительно разорвал сущес

Итерация: 434000. Validation loss: 0.002421342534944415. Validation f1_score: 0.9467874765396118
Итерация: 435000. Validation loss: 0.005437033250927925. Validation f1_score: 0.9488938450813293


Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\evstr\anaconda3\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\evstr\anaconda3\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] Удаленный хост принудительно разорвал существующее подключение

KeyboardInterrupt

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\Users\evstr\anaconda3\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\evstr\anaconda3\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] Удаленный хост принудит